In [1]:
import sys
import time
import datetime
import gc
import pandas as pd
import numpy as np
from typing import Callable
from joblib import Parallel, delayed
from mlxtend.frequent_patterns import apriori
from utlis import *
import gurobipy as grb
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data_path(filename:str) -> str:
    return str(os.path.dirname(os.getcwd()))+'\\data\\'+filename+'.csv'


def get_time_list(starttime:datetime.datetime,endtime:datetime.datetime):
    time_list = []
    while starttime<=endtime:
        time_list.append(starttime)
        starttime += datetime.timedelta(days=1)
    return time_list


def str_to_datetime(x:str) -> datetime.datetime:
    return datetime.datetime.strptime(x, "%Y-%m-%d")


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if col == 'date':
            continue   
        elif col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    # df[col] = df[col].astype(np.float16)
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # df[col] = df[col].astype('category')
            pass
    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# 根据历史订单销售数据判定sku类型
sku_sales_df = pd.read_csv(get_data_path('sku_sales'), sep=',', encoding='utf8')
sku_sales_df['order_time'] = sku_sales_df.apply(func=lambda x: x['order_time'][:10], axis=1)
sku_sales_df = sku_sales_df.rename(columns={'order_time':'date'})
sku_sales_df['date'] = sku_sales_df['date'].apply(str_to_datetime)

# 零售品
not_complete_list = sku_sales_df.loc[(sku_sales_df.quantity%1!=0),'sku_id'].unique().tolist()
print("零售品列表如下：",not_complete_list)
print(f"零售品数量为{len(not_complete_list)}")

# 预测期每个店铺的停售品
not_sale_store_list = []
for i in range(1,13):
    print(f"第{i}号门店的停售品为：")
    all_sku_set = set([i for i in range(1,1001)])
    condition1 = (sku_sales_df.date>=datetime.datetime(2022,8,15))
    condition2 = (sku_sales_df.date<=datetime.datetime(2022,9,30)) 
    condition3 = (sku_sales_df.date>=datetime.datetime(2023,8,10))
    condition4 = (sku_sales_df.store_id==i)
    all_sku_set = set([_ for _ in range(1,1001)])
    sale_set = set(sku_sales_df.loc[((condition1&condition2) | condition3) & (condition4),'sku_id'].unique().tolist())
    not_sale_list = list(all_sku_set-sale_set)
    print(not_sale_list)
    print(f"停售种类数量为{len(not_sale_list)}")
    not_sale_store_list.append(not_sale_list)

# 新品（历史前一个月无销售记录）
new_sale_list = []
history_sale_set = set(sku_sales_df.loc[(sku_sales_df.date<datetime.datetime(2023,8,1)),'sku_id'].unique().tolist())
new_set = set(sku_sales_df.loc[(sku_sales_df.date>=datetime.datetime(2023,8,1)),'sku_id'].unique().tolist())
new_sale_list = [_ for  _ in new_set if _ not in history_sale_set]
print("新品列表如下：",new_sale_list)
print(f"新品数量为{len(new_sale_list)}")


# sku销量规模0-2,3-7,8-15,16-inf
# 只看历史同期数据
condition1 = (sku_sales_df.date>=datetime.datetime(2022,8,25))
condition2 = (sku_sales_df.date<=datetime.datetime(2022,9,20)) 
condition3 = (sku_sales_df.date>=datetime.datetime(2023,8,15))
sku_sales_df = sku_sales_df[(condition1 & condition2) | condition3]
online_sku_first_sale_list = []
online_sku_second_sale_list = []
online_sku_third_sale_list = []
online_sku_forth_sale_list = []
online_sku_sales_df = sku_sales_df[sku_sales_df.channel==2]
online_sku_sales_df = online_sku_sales_df.groupby(['store_id','sku_id','date'])[['order_id','quantity']].agg({'order_id':'count','quantity':'sum'}).reset_index()
online_sku_sales_df = online_sku_sales_df.groupby('sku_id')['quantity'].median().reset_index()
online_sku_sales_df = online_sku_sales_df.sort_values(by='sku_id')
tmp_list = online_sku_sales_df.sku_id.tolist()
for i in tmp_list:
    if online_sku_sales_df.loc[(online_sku_sales_df['sku_id']==i),'quantity'].values<=2:
        online_sku_first_sale_list.append(i)
    elif online_sku_sales_df.loc[(online_sku_sales_df['sku_id']==i),'quantity'].values<=7:
        online_sku_second_sale_list.append(i)
    elif online_sku_sales_df.loc[(online_sku_sales_df['sku_id']==i),'quantity'].values<=15:
        online_sku_third_sale_list.append(i)
    else:
        online_sku_forth_sale_list.append(i)
print(f"线上订单销量中位数小于2的sku种类数量为{len(online_sku_first_sale_list)}")
print(f"线上订单销量中位数小于7的sku种类数量为{len(online_sku_second_sale_list)}")
print(f"线上订单销量中位数小于15的sku种类数量为{len(online_sku_third_sale_list)}")
print(f"线上订单销量中位数大于15的sku种类数量为{len(online_sku_forth_sale_list)}")
offline_sku_first_sale_list = []
offline_sku_second_sale_list = []
offline_sku_third_sale_list = []
offline_sku_forth_sale_list = []
offline_sku_sales_df = sku_sales_df[sku_sales_df.channel==1]
offline_sku_sales_df = offline_sku_sales_df.groupby(['store_id','sku_id','date'])[['order_id','quantity']].agg({'order_id':'count','quantity':'sum'}).reset_index()
offline_sku_sales_df = offline_sku_sales_df.groupby('sku_id')['quantity'].median().reset_index()
offline_sku_sales_df = offline_sku_sales_df.sort_values(by='sku_id')
tmp_list = offline_sku_sales_df.sku_id.tolist()
for i in tmp_list:
        if offline_sku_sales_df.loc[(offline_sku_sales_df['sku_id']==i),'quantity'].values<=2:
            offline_sku_first_sale_list.append(i)
        elif offline_sku_sales_df.loc[(offline_sku_sales_df.sku_id==i),'quantity'].values<=7:
            offline_sku_second_sale_list.append(i)
        elif offline_sku_sales_df.loc[(offline_sku_sales_df.sku_id==i),'quantity'].values<=15:
            offline_sku_third_sale_list.append(i)
        else:
            offline_sku_forth_sale_list.append(i)
print(f"线下订单销量中位数小于2的sku种类数量为{len(offline_sku_first_sale_list)}")
print(f"线下订单销量中位数小于7的sku种类数量为{len(offline_sku_second_sale_list)}")
print(f"线下订单销量中位数小于15的sku种类数量为{len(offline_sku_third_sale_list)}")
print(f"线下订单销量中位数大于15的sku种类数量为{len(offline_sku_forth_sale_list)}")

零售品列表如下： [14, 259, 44, 442, 150, 449, 448, 443]
零售品数量为8
第1号门店的停售品为：
[867, 580, 713, 905, 906, 652, 127, 653, 814, 784, 721, 907, 909, 922, 572, 350, 543]
停售种类数量为17
第2号门店的停售品为：
[865, 866, 451, 580, 8, 713, 586, 936, 652, 653, 693, 601, 350, 543]
停售种类数量为14
第3号门店的停售品为：
[8, 905, 906, 907, 653, 909, 786, 791, 543, 931, 936, 446, 451, 580, 713, 720, 721, 350, 865, 867]
停售种类数量为20
第4号门店的停售品为：
[865, 451, 580, 936, 713, 652, 721, 601, 921, 956, 350, 543]
停售种类数量为12
第5号门店的停售品为：
[865, 866, 995, 580, 8, 713, 586, 905, 906, 907, 909, 956, 569, 127, 350, 543]
停售种类数量为16
第6号门店的停售品为：
[451, 580, 867, 713, 652, 350, 543]
停售种类数量为7
第7号门店的停售品为：
[865, 451, 580, 867, 868, 713, 652, 366, 785, 601, 350, 543]
停售种类数量为12
第8号门店的停售品为：
[865, 451, 580, 867, 931, 713, 586, 652, 653, 785, 958, 309, 601, 795, 956, 350, 543]
停售种类数量为17
第9号门店的停售品为：
[865, 451, 580, 934, 713, 652, 366, 721, 786, 309, 956, 350]
停售种类数量为12
第10号门店的停售品为：
[896, 8, 905, 906, 907, 652, 909, 910, 785, 923, 543, 927, 931, 184, 956, 451, 580, 713, 586, 72

In [41]:
# 查看过去2周的销量表现
sku_sales_df_2 = sku_sales_df[(sku_sales_df.date>=datetime.datetime(2023,8,18))]
print(sku_sales_df_2.groupby(['date'])['quantity'].sum())

date
2023-08-18    56114.0
2023-08-19    74615.5
2023-08-20    71599.5
2023-08-21    51528.5
2023-08-22    51823.1
2023-08-23    52322.8
2023-08-24    51058.2
2023-08-25    64153.6
2023-08-26    81155.6
2023-08-27    74674.7
2023-08-28    56045.8
2023-08-29    54488.3
2023-08-30    58566.1
2023-08-31    56938.6
Name: quantity, dtype: float64


In [43]:
# 读取数据
# 销量预测数据
predict_df = pd.read_csv(get_data_path('Base_stacking3_predict_df_clf_4'))
predict_df['date'] = predict_df['date'].apply(str_to_datetime)

# 关联品数据(待更新)
apriori_df = pd.read_csv(get_data_path('apriori_df_4'))

# 合并数据
predict_df = pd.merge(left=predict_df,right=apriori_df,how='left',on=['store_id','sku_id'])
predict_df

,date,y_online,y_offline,y_all,store_id,sku_id,if_selected_possible,if_high_value,online_order_mid_quantity,online_17d_avg_order_num,online_7d_sale_days,online_inventory_level,online_inventory_plenty_unit_ceil,online_inventory_plenty_unit_floor,offline_order_mid_quantity,offline_17d_avg_order_num,offline_7d_sale_days,offline_inventory_level,offline_inventory_plenty_unit_ceil,offline_inventory_plenty_unit_floor
0,2023-09-01,0.0,2.0,2.0,1,1,0.0,0.0,1.0,1.00,3.0,0,1.0,0.0,1.0,2.40,4.0,0,1.0,0.0
1,2023-09-01,1.0,3.0,4.0,1,2,1.0,1.0,2.0,1.27,4.0,0,1.0,0.0,1.0,2.27,6.0,1,1.0,0.0
2,2023-09-01,3.0,4.0,7.0,1,3,1.0,1.0,1.0,2.25,6.0,1,1.0,0.0,1.0,3.07,6.0,1,1.0,0.0
3,2023-09-01,2.0,0.0,2.0,1,4,0.0,0.0,2.0,1.33,1.0,0,1.0,0.0,1.0,1.57,3.0,0,1.0,0.0
4,2023-09-01,7.0,30.0,37.0,1,5,1.0,1.0,2.0,1.67,6.0,1,1.0,0.0,1.0,12.53,7.0,1,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167995,2023-09-14,2.0,3.0,5.0,12,996,0.0,0.0,1.0,1.50,1.0,0,1.0,0.0,1.0,1.50,2.0,0,1.0,0.0
167996,2023-09-14,3.0,2.0,5.0,12,997,0.0,0.0,1.0,1.33,3.0,0,1.0,0.0,1.0,1.00,6.0,1,1.0,0.0
167997,2023-09-14,2.0,1.0,3.0,12,998,0.0,0.0,1.0,1.60,3.0,0,1.0,0.0,1.0,1.80,2.0,0,1.0,0.0
167998,2023-09-14,2.0,2.0,4.0,12,999,0.0,0.0,1.0,1.86,3.0,0,1.0,0.0,1.0,2.08,7.0,1,1.0,0.0


In [47]:
# 查看预测数据的预测情况
print("=="*10)
print("总销量预测如下：")
print(predict_df.groupby(['date'])['y_all'].sum())
# print("=="*10)
# print("线上订单销量预测如下：")
# print(predict_df.groupby(['date'])['y_online'].sum())
# print("=="*10)
# print("线下订单销量预测如下：")
# print(predict_df.groupby(['date'])['y_offline'].sum())

总销量预测如下：
date
2023-09-01     92787.4
2023-09-02    112848.1
2023-09-03    100658.4
2023-09-04     79062.0
2023-09-05     76608.5
2023-09-06     82425.8
2023-09-07     80558.8
2023-09-08    100205.7
2023-09-09    119542.1
2023-09-10    105941.3
2023-09-11     83356.3
2023-09-12     80312.3
2023-09-13     86522.3
2023-09-14     84095.0
Name: y_all, dtype: float64


In [48]:
# 读取数据可能存在浮点数问题,这里保险起见，重新得到y_all
predict_df['y_all'] = predict_df['y_online']+predict_df['y_offline']

# 业务处理停售品
for i in range(len(not_sale_store_list)):
    for j in not_sale_store_list[i]:
        for col in ['y_online','y_offline','y_all']:
            predict_df.loc[((predict_df.store_id==i+1) & (predict_df.sku_id==j)),col] = 0

# 预测周期
predict_start_date = datetime.datetime(2023,9,1) # 预测开始时间
predict_end_date = datetime.datetime(2023,9,14)  # 预测结束时间
predict_periods = 14 # 预测步长
pred_date_list = [] # 预测日期列表
tmp = predict_start_date
while tmp<=predict_end_date:
    pred_date_list.append(tmp)
    tmp += datetime.timedelta(days=1)

In [76]:
kpi_df_list = []
for t in range(14):
    for s in range(1,13):
        print(f"正在分配第{t+1}天第{s}店铺的前后场备货量......")
        predict_df_t_i = predict_df[(predict_df.date==pred_date_list[t]) & (predict_df.store_id==s)]

        offline_y_t_i_list = predict_df_t_i['y_offline'].tolist()
        online_y_t_i_list = predict_df_t_i['y_online'].tolist()
        all_y_t_i_list = predict_df_t_i['y_all'].tolist()
        if_selected_possible_t_i_list = predict_df_t_i['if_selected_possible'].tolist()
        if_high_value_t_i_list = predict_df_t_i['if_high_value'].tolist()
        online_order_mid_quantity_t_i_list = predict_df_t_i['online_order_mid_quantity'].tolist()
        offline_order_mid_quantity_t_i_list = predict_df_t_i['offline_order_mid_quantity'].tolist()
        online_inventory_level_t_i_list = predict_df_t_i['online_inventory_level'].tolist()
        offline_inventory_level_t_i_list = predict_df_t_i['offline_inventory_level'].tolist()
        online_17d_avg_order_num_t_i_list = predict_df_t_i['online_17d_avg_order_num'].tolist()
        offline_17d_avg_order_num_t_i_list = predict_df_t_i['offline_17d_avg_order_num'].tolist()

        # 周末，安全库存线上和线下均采用乐观估计，非周末的话线下则采用保守估计，线上采用乐观估计?
        if t in (1,2,8,9):
            online_inventory_plenty_unit_t_i_list = predict_df_t_i['online_inventory_plenty_unit_ceil'].tolist()
            offline_inventory_plenty_unit_t_i_list = predict_df_t_i['offline_inventory_plenty_unit_ceil'].tolist()
        else:
            # online_inventory_plenty_unit_t_i_list = predict_df_t_i['online_inventory_plenty_unit_floor'].tolist()
            online_inventory_plenty_unit_t_i_list = predict_df_t_i['online_inventory_plenty_unit_ceil'].tolist()
            offline_inventory_plenty_unit_t_i_list = predict_df_t_i['offline_inventory_plenty_unit_floor'].tolist()
        

        all_y_sum = sum(all_y_t_i_list)
        # 高支持度sku额外保持一笔线上订单能力
        all_selected_possible_sum = sum([if_selected_possible_t_i_list[i]*online_order_mid_quantity_t_i_list[i] for i in range(1000)])
        # 高关联度sku额外保持一笔线上订单能力
        all_high_value_sum = sum([if_high_value_t_i_list[i]*online_order_mid_quantity_t_i_list[i] for i in range(1000)])
        # 安全库存量
        online_inventory_quantity_sum_t_i_list = [online_inventory_level_t_i_list[i]*online_inventory_plenty_unit_t_i_list[i]*online_order_mid_quantity_t_i_list[i] for i in range(1000)]
        online_inventory_quantity_sum = sum(online_inventory_quantity_sum_t_i_list)
        offline_inventory_quantity_sum_t_i_list = [offline_inventory_level_t_i_list[i]*offline_inventory_plenty_unit_t_i_list[i]*offline_order_mid_quantity_t_i_list[i] for i in range(1000)]
        offline_inventory_quantity_sum = sum(offline_inventory_quantity_sum_t_i_list)

        # 默认后场关联参数(0901/0902/0903/0908/0909/0910则放大)
        alpha_support = 1
        alpha_social = 1
        # 周五/六/日
        if t in (0,1,2,7,8,9):
            alpha_social = 2
        
        # 创建模型
        model = grb.Model(name=f"dt{t+1}_store{s}_model")
        model.setParam('OutputFlag', 0) # 控制台不输出
        # 更新模型
        model.update()
        # model.setParam('MIPGap',0.05)
        # model.setParam('Timelimit',120)
        # 添加变量
        # 创建决策变量
        '''
        F_i: 前场是否备货
        B_i: 后场是否备货
        XF_i: 前场备货量
        XB_i: 后场备货量
        '''
        F_variables = {}
        B_variables = {}
        XF_variables = {}
        XB_variables = {}
        for i in range(1000):
            F_variables[i] = model.addVar(vtype=grb.GRB.BINARY)
            B_variables[i] = model.addVar(vtype=grb.GRB.BINARY)
            XF_variables[i] = model.addVar(vtype=grb.GRB.CONTINUOUS,lb=0)
            XB_variables[i] = model.addVar(vtype=grb.GRB.CONTINUOUS,lb=0)
        # 中间变量
        O = model.addVar(vtype=grb.GRB.CONTINUOUS,lb=0)
        # 创建约束
        '''1.后场备货种类小于所有备货种类的20%'''
        model.addConstr(grb.quicksum(B_variables[i] for i in range(1000)) <= 200,
                                    name="Con1")
        '''2.后场备货数量小于所有备货数量的40%'''
        model.addConstr(grb.quicksum(XB_variables[i] for i in range(1000)) <= 0.4*(all_y_sum + alpha_support*all_selected_possible_sum + 
                                                                                   alpha_social*all_high_value_sum + 
                                                                                   online_inventory_quantity_sum +
                                                                                   offline_inventory_quantity_sum),
                                    name="Con2")
        '''3.在准确预测的假设下,保证履约约束'''
        for i in range(1000):
            model.addConstr(XF_variables[i]+XB_variables[i]==(online_y_t_i_list[i]+offline_y_t_i_list[i] + 
                                                              alpha_support*if_selected_possible_t_i_list[i]*online_order_mid_quantity_t_i_list[i] +
                                                              alpha_social*if_high_value_t_i_list[i]*online_order_mid_quantity_t_i_list[i] +
                                                              online_inventory_quantity_sum_t_i_list[i] +
                                                              offline_inventory_quantity_sum_t_i_list[i]),
                                    name=f"Con3_sku{str(i+1)}")
        '''4.在可售情况下保存前场备货量大于0'''
        for i in range(1000):
            if (online_y_t_i_list[i]+offline_y_t_i_list[i]+alpha_support*if_selected_possible_t_i_list[i]*online_order_mid_quantity_t_i_list[i]
                +alpha_social*if_high_value_t_i_list[i]*online_order_mid_quantity_t_i_list[i]+online_inventory_quantity_sum_t_i_list[i]+
                offline_inventory_quantity_sum_t_i_list[i])>0:
                if i+1 in not_complete_list:
                    model.addConstr(F_variables[i]==1, name=f"Con4_sku{str(i+1)}_1")
                    model.addConstr(XF_variables[i]>=0.1, name=f"Con4_sku{str(i+1)}_2")
                else:
                    model.addConstr(F_variables[i]==1, name=f"Con4_sku{str(i+1)}_1")
                    model.addConstr(XF_variables[i]>=1, name=f"Con4_sku{str(i+1)}_2")
        '''5.后场备货量小于预测销量+乐观估计量+线上库存量'''
        for i in range(1000):
            model.addConstr(XB_variables[i]<=(online_y_t_i_list[i] +
                                              alpha_support*if_selected_possible_t_i_list[i]*online_order_mid_quantity_t_i_list[i] +
                                              alpha_social*if_high_value_t_i_list[i]*online_order_mid_quantity_t_i_list[i] +
                                              online_inventory_quantity_sum_t_i_list[i]),
                            name=f"Con5_sku{str(i+1)}")
        '''6.备货决策和备货量的指示关系约束'''
        for i in range(1000):
            model.addConstr(XB_variables[i]<=B_variables[i]*100000,
                            name=f"Con6_ind{str(i+1)}_1")
            model.addConstr(XF_variables[i]<=F_variables[i]*100000,
                            name=f"Con6_ind{str(i+1)}_2")
        '''7.是否后场可选的关系约束'''
        for i in range(1000):
            model.addConstr(B_variables[i]<=if_selected_possible_t_i_list[i],
                            name=f"Con7_sku{str(i+1)}")
        '''8.中间变量约束'''
        '''后场效率主要是看订单完成次数，而不是单独的订单完成量，因此对次数和量加权，次数和量的量级约为7:1，因此权重取0.9和0.1'''
        # model.addConstr(O == grb.quicksum(XB_variables[i] for i in range(1000)),
        #                             name="Con8")
        # model.addConstr(O == grb.quicksum(0.9*B_variables[i]*online_17d_avg_order_num_t_i_list[i] + 
        #                                   0.1*XB_variables[i] for i in range(1000)),
        #                             name="Con8")
        model.addConstr(O == grb.quicksum(0.9*B_variables[i]*online_17d_avg_order_num_t_i_list[i]*XB_variables[i]/(online_y_t_i_list[i]+1) 
                                          + 0.1*XB_variables[i] for i in range(1000)),
                                name="Con8")
    
        # 目标优化,默认最小化方向
        model.setObjective(O, grb.GRB.MAXIMIZE)
        # 执行线性规划模型
        model.optimize()
        
        x_k = []
        x_m = []
        B_list = []
        B_ord_list = []
        for i in range(1000):
            if i+1 in not_complete_list:
                x_k.append(round(XF_variables[i].X,1))
                x_m.append(round(XB_variables[i].X,1))
            else:
                x_k.append(round(XF_variables[i].X,0))
                x_m.append(round(XB_variables[i].X,0))
            B_list.append(round(B_variables[i].X,0))
            num_i = (online_y_t_i_list[i] + alpha_support*if_selected_possible_t_i_list[i]*online_order_mid_quantity_t_i_list[i] +
                     alpha_social*if_high_value_t_i_list[i]*online_order_mid_quantity_t_i_list[i] + online_inventory_quantity_sum_t_i_list[i])
            if num_i!=0:
                B_ord_list.append(online_17d_avg_order_num_t_i_list[i]*B_list[i]*x_m[i]/(num_i))
            else:
                B_ord_list.append(0)
        
        print(f"经模型求解，第{t+1}天第{s}个店铺的后场总备货量为{round(sum(x_k)+sum(x_m))},后场备货种类为{round(sum(B_list))},后场完成订单数为{round(sum(B_ord_list))}")
        
        predict_df_t_i['x_k'] = x_k
        predict_df_t_i['x_m'] = x_m
        # 由于浮点数问题，可能后场备货量超过40%，此处修正，将不足的部分随机补充到一个随机sku的前场中
        x_m_sum =  predict_df_t_i['x_m'].sum()
        x_all_sum = predict_df_t_i['x_m'].sum()+predict_df_t_i['x_k'].sum()
        while round(x_m_sum,2) >= round(x_all_sum*0.4,2):
            random_sku = np.random.randint(1,1001)
            predict_df_t_i.loc[predict_df_t_i.sku_id==random_sku,'x_k'] += 1
            x_m_sum =  predict_df_t_i['x_m'].sum()
            x_all_sum = predict_df_t_i['x_m'].sum()+predict_df_t_i['x_k'].sum()
        
        kpi_df_list.append(predict_df_t_i)
kpi_df = pd.concat(kpi_df_list)

正在分配第1天第1店铺的前后场备货量......
经模型求解，第1天第1个店铺的后场总备货量为7202,后场备货种类为200,后场完成订单数为1084
正在分配第1天第2店铺的前后场备货量......
经模型求解，第1天第2个店铺的后场总备货量为8342,后场备货种类为200,后场完成订单数为1248
正在分配第1天第3店铺的前后场备货量......
经模型求解，第1天第3个店铺的后场总备货量为7397,后场备货种类为200,后场完成订单数为1234
正在分配第1天第4店铺的前后场备货量......
经模型求解，第1天第4个店铺的后场总备货量为10510,后场备货种类为200,后场完成订单数为1896
正在分配第1天第5店铺的前后场备货量......
经模型求解，第1天第5个店铺的后场总备货量为5563,后场备货种类为200,后场完成订单数为760
正在分配第1天第6店铺的前后场备货量......
经模型求解，第1天第6个店铺的后场总备货量为15513,后场备货种类为200,后场完成订单数为2924
正在分配第1天第7店铺的前后场备货量......
经模型求解，第1天第7个店铺的后场总备货量为10822,后场备货种类为200,后场完成订单数为1990
正在分配第1天第8店铺的前后场备货量......
经模型求解，第1天第8个店铺的后场总备货量为9308,后场备货种类为200,后场完成订单数为1697
正在分配第1天第9店铺的前后场备货量......
经模型求解，第1天第9个店铺的后场总备货量为9386,后场备货种类为200,后场完成订单数为1732
正在分配第1天第10店铺的前后场备货量......
经模型求解，第1天第10个店铺的后场总备货量为8405,后场备货种类为200,后场完成订单数为1565
正在分配第1天第11店铺的前后场备货量......
经模型求解，第1天第11个店铺的后场总备货量为10286,后场备货种类为200,后场完成订单数为2069
正在分配第1天第12店铺的前后场备货量......
经模型求解，第1天第12个店铺的后场总备货量为13063,后场备货种类为200,后场完成订单数为2377
正在分配第2天第1店铺的前后场备货量......
经模型求解，第2天第1个店铺的后场总备货量为9425,后场备货种类为200,后场完成订单数为1088
正在

In [77]:
test_result = kpi_df[['date','store_id','sku_id','x_k','x_m']]
test_result.to_csv(get_data_path('test_result_clf_final'),index=False)

In [78]:
test_result

,date,store_id,sku_id,x_k,x_m
0,2023-09-01,1,1,2.0,0.0
1,2023-09-01,1,2,3.0,7.0
2,2023-09-01,1,3,4.0,7.0
3,2023-09-01,1,4,2.0,0.0
4,2023-09-01,1,5,32.0,15.0
...,...,...,...,...,...
167995,2023-09-14,12,996,5.0,0.0
167996,2023-09-14,12,997,5.0,0.0
167997,2023-09-14,12,998,3.0,0.0
167998,2023-09-14,12,999,4.0,0.0


In [79]:
# 检验后场量是否满足要求
for t in range(14):
    for s in range(1,13):
        watch_df = test_result[(test_result.date==pred_date_list[t]) & (test_result.store_id==s)]
        if round(watch_df.x_m.sum(),2)>=round((watch_df.x_m.sum()+watch_df.x_k.sum())*0.4,2):
            print(f"9月{t+1}号{s}号店铺后场备货量为{watch_df.x_m.sum()},全场备货量的40%为{(watch_df.x_m.sum()+watch_df.x_k.sum())*0.4},不符要求")
print("No error!")

No error!


In [80]:
# 检查是否有小于0的备货量
print(test_result.x_k.min(),test_result.x_k.max(),test_result.x_m.min(),test_result.x_m.max())

0.0 553.5 0.0 551.5


In [83]:
# 检查是否存在空值
test_result.isna().any().sum()

0

In [86]:
# 检查前场陈列
illegal_df = test_result[(test_result['x_k'] <= 0) & (test_result['x_m'] > 0)]
if len(illegal_df) > 0:
    for row_id, illegal_obj in illegal_df.iterrows():
        sku, store, date = illegal_obj.sku_id, illegal_obj.store_id, illegal_obj.date
        x_k, x_m = illegal_obj.x_k, illegal_obj.x_m
        err_msg = "[规则检查] 前场陈列不符合赛题要求！如：[商品{},门店{},日期{}][备货数量:后{}>0, 前{}=0]".format(sku, store, date,
                                                                                    x_m, x_k)
        print(err_msg)
print("No error!")

No error!


In [88]:
# 检查后场库存
class Param(object):
    """
    参数类
    """
    neg_inf = -99999999999  # 不能用np.inf，防止下游解析出问题
    front_stock_tol = 1e-4  # 前场库存容差
    kpi_ndigits = 4  # kpi小数点有效位数
    result_ndigits = 2  # 提交结果小数点有效位数
    kpi_columns = ['store_id', 'date', '订单履约率A', '净利润S', '履约效率D', '销售利润R', '缺货机会成本L', '损耗成本C', '履约成本V', \
                   '搬运成本V1', '拣货成本V2', '可履约订单量n_F', '可履约线上订单量n_F_online', '不可履约订单量n_N', '后场完单量n_B', \
                   '后场拣货种类k_bp', '后场拣货数量c_bp', '前场拣货种类k_fp', '前场拣货数量c_fp', '后场向前场搬货种类k_bfc', '后场向前场搬货数量c_bfc']

    result_columns = ['date', 'store_id', 'sku_id', 'x_k', 'x_m']

    back_sku_stock_cnt_ratio_upper = 0.4  # 后场备货商品数量占门店全部备货量的比例 <= 0.4
    back_sku_kind_cnt_ratio_upper = 0.2  # 后场备货商品种类占门店全部商品种类的比例 <= 0.2
    avg_fulfill_ratio_lower = 0.75  # 平均订单履约满足率
    wastage_ratio = 0.3  # 报损比例0.3，门店j，第t日前后场当天剩余未售出的商品数量的30%按照进货成本价全部报损，其他剩余的库存直接清零处理。


err_msg = ""
kind_ratio = Param.back_sku_kind_cnt_ratio_upper
stock_ratio = Param.back_sku_stock_cnt_ratio_upper

# 总备货种类和数量，后场备货数量
stock_df = test_result[['date', 'store_id', 'sku_id', 'x_k', 'x_m']].groupby(
    ['date', 'store_id']).agg({'sku_id': 'nunique', 'x_k': 'sum', 'x_m': 'sum'}).reset_index()
stock_df = stock_df.rename(columns={'sku_id': 'kind_cnt', 'x_k': 'front_stock_cnt',
                                    'x_m': 'back_stock_cnt'})
stock_df['stock_cnt'] = stock_df['front_stock_cnt'] + stock_df['back_stock_cnt']

# 后场最多摆放sku种类和数量，参赛选手会输出所有sku，即使补货为0
stock_df['max_back_sku_kind_cnt'] = stock_df['kind_cnt'] * kind_ratio
stock_df['max_back_sku_stock_cnt'] = stock_df['stock_cnt'] * stock_ratio

# 后场备货种类
back_stock_df = test_result[test_result['x_m'] > 0]
back_stock_df = back_stock_df[['date', 'store_id', 'sku_id']].groupby(['date', 'store_id']).agg(
    {'sku_id': 'nunique'}).reset_index()
back_stock_df = back_stock_df.rename(columns={'sku_id': 'back_kind_cnt'})

# 合并
stock_df = pd.merge(left=stock_df, right=back_stock_df, how='left', on=['date', 'store_id'])

# 检查
stock_df['back_sku_kind_cnt_diff'] = stock_df['max_back_sku_kind_cnt'] - stock_df['back_kind_cnt']
stock_df['back_sku_stock_cnt_diff'] = stock_df['max_back_sku_stock_cnt'] - stock_df['back_stock_cnt']

# 筛选出不合法（date, store）
illegal_df = stock_df[
    (stock_df['back_sku_kind_cnt_diff'] < 0) | (stock_df['back_sku_stock_cnt_diff'] < 0)]

if len(illegal_df) > 0:
    for row_id, illegal_obj in illegal_df.iterrows():
        store, date = illegal_obj.store_id, illegal_obj.date
        back_kind_cnt, back_stock_cnt = illegal_obj.back_kind_cnt, illegal_obj.back_stock_cnt
        kind_cnt, stock_cnt = illegal_obj.kind_cnt, illegal_obj.stock_cnt
        if back_kind_cnt > kind_cnt * kind_ratio:
            err_msg = "[规则检查] 后场库存不符合赛题要求！如：[门店{},日期{},后场备货种类{}超过总备货种类{}*{}={}]" \
                .format(store, date, back_kind_cnt, kind_cnt, kind_ratio,
                        round(kind_cnt * kind_ratio, Param.kpi_ndigits))
            print(err_msg)

        if back_stock_cnt > stock_cnt * stock_ratio:
            err_msg = "[规则检查] 后场库存不符合赛题要求！如：[门店{},日期{},后场备货数量{}超过总备货数量{}*{}={}]" \
                .format(store, date, round(back_stock_cnt, Param.kpi_ndigits),
                        round(stock_cnt, Param.kpi_ndigits), stock_ratio,
                        round(stock_cnt * stock_ratio, Param.kpi_ndigits))
            print(err_msg)

        err_msg = "[规则检查] 后场库存不符合赛题要求！"
        print('err_msg')
print("No error")


No error


In [81]:
# 查看备货量是否合理
test_result['x'] = test_result['x_k'] + test_result['x_m']
print("=="*10)
print("总备货量如下：")
print(test_result.groupby(['date'])['x'].sum())
print("=="*10)
print("线上订单备货量如下：")
print(test_result.groupby(['date'])['x_m'].sum())
print("=="*10)
print("线下订单备货量如下：")
print(test_result.groupby(['date'])['x_k'].sum())

总备货量如下：
date
2023-09-01    115801.4
2023-09-02    140684.6
2023-09-03    128497.9
2023-09-04     97844.3
2023-09-05     95391.8
2023-09-06    101210.1
2023-09-07     99345.1
2023-09-08    123220.7
2023-09-09    147378.6
2023-09-10    133780.8
2023-09-11    102136.6
2023-09-12     99096.6
2023-09-13    105303.6
2023-09-14    102878.3
Name: x, dtype: float64
线上订单备货量如下：
date
2023-09-01    46295.8
2023-09-02    55540.0
2023-09-03    50223.4
2023-09-04    39063.0
2023-09-05    38080.1
2023-09-06    40303.6
2023-09-07    39257.5
2023-09-08    48905.8
2023-09-09    57178.3
2023-09-10    51379.9
2023-09-11    40513.7
2023-09-12    39265.6
2023-09-13    41733.3
2023-09-14    40367.8
Name: x_m, dtype: float64
线下订单备货量如下：
date
2023-09-01    69505.6
2023-09-02    85144.6
2023-09-03    78274.5
2023-09-04    58781.3
2023-09-05    57311.7
2023-09-06    60906.5
2023-09-07    60087.6
2023-09-08    74314.9
2023-09-09    90200.3
2023-09-10    82400.9
2023-09-11    61622.9
2023-09-12    59831.0
2023-09-13 

In [82]:
watch_df = pd.merge(left=test_result,right=apriori_df,on=['store_id','sku_id'],how='left')
watch_df

,date,store_id,sku_id,x_k,x_m,x,if_selected_possible,if_high_value,online_order_mid_quantity,online_17d_avg_order_num,online_7d_sale_days,online_inventory_level,online_inventory_plenty_unit_ceil,online_inventory_plenty_unit_floor,offline_order_mid_quantity,offline_17d_avg_order_num,offline_7d_sale_days,offline_inventory_level,offline_inventory_plenty_unit_ceil,offline_inventory_plenty_unit_floor
0,2023-09-01,1,1,2.0,0.0,2.0,0.0,0.0,1.0,1.00,3.0,0,1.0,0.0,1.0,2.40,4.0,0,1.0,0.0
1,2023-09-01,1,2,3.0,7.0,10.0,1.0,1.0,2.0,1.27,4.0,0,1.0,0.0,1.0,2.27,6.0,1,1.0,0.0
2,2023-09-01,1,3,4.0,7.0,11.0,1.0,1.0,1.0,2.25,6.0,1,1.0,0.0,1.0,3.07,6.0,1,1.0,0.0
3,2023-09-01,1,4,2.0,0.0,2.0,0.0,0.0,2.0,1.33,1.0,0,1.0,0.0,1.0,1.57,3.0,0,1.0,0.0
4,2023-09-01,1,5,32.0,15.0,47.0,1.0,1.0,2.0,1.67,6.0,1,1.0,0.0,1.0,12.53,7.0,1,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167995,2023-09-14,12,996,5.0,0.0,5.0,0.0,0.0,1.0,1.50,1.0,0,1.0,0.0,1.0,1.50,2.0,0,1.0,0.0
167996,2023-09-14,12,997,5.0,0.0,5.0,0.0,0.0,1.0,1.33,3.0,0,1.0,0.0,1.0,1.00,6.0,1,1.0,0.0
167997,2023-09-14,12,998,3.0,0.0,3.0,0.0,0.0,1.0,1.60,3.0,0,1.0,0.0,1.0,1.80,2.0,0,1.0,0.0
167998,2023-09-14,12,999,4.0,0.0,4.0,0.0,0.0,1.0,1.86,3.0,0,1.0,0.0,1.0,2.08,7.0,1,1.0,0.0
